In [1]:
import glob
import datetime
import numpy as np
import pandas as pd
from library.preprocessing import distance

In [18]:
all_folders=glob.glob("./Trails/*/*/")

test_dates=['2019-06-28','2019-06-30','2019-07-01','2019-07-02','2019-07-03','2019-07-04','2019-07-05']
formatted_test_dates=['-'.join(d.split("-")[::-1]) for d in test_dates] #reversing order

get_date=lambda fname:fname.split("\\")[-2].split("_")[0]
get_format_date=lambda fname: '-'.join(get_date(fname).split("-")[::-1])

#train folders
train_file_pairs=\
[(f+"GPS.csv",
 f+"mu_serv_{}.csv".format(get_format_date(f))) for f in all_folders if get_date(f) in formatted_test_dates]

In [19]:
def get_average_speed_in_trail_with_tz(gps,mu):
    total_distance=0
    latlong=gps[['#lat','long']]
    for i in range(latlong.shape[0]-1):
        total_distance+=distance(*latlong.iloc[i].values.tolist(),*latlong.iloc[i+1].values.tolist())

    travel_time=\
    (pd.to_datetime(gps.iloc[-1].time).timestamp()-pd.to_datetime(gps.iloc[0].time).timestamp())-mu.stay_duration.sum()

    avg_speed=total_distance/travel_time
    return [avg_speed,mu.time_zone[0]]

# Loop through training set

In [20]:
speed_with_tz=[]

for gps_file,mu_file in train_file_pairs:
    gps=pd.read_csv(gps_file)
    mu=pd.read_csv(mu_file)
    speed_with_tz.append(get_average_speed_in_trail_with_tz(gps,mu))
    print("ON file ",gps_file)

ON file  ./Trails\down\01-07-2019\GPS.csv
ON file  ./Trails\down\01-07-2019_DATA_17_03_44\GPS.csv
ON file  ./Trails\down\02-07-2019\GPS.csv
ON file  ./Trails\down\02-07-2019_DATA_09_52_39\GPS.csv
ON file  ./Trails\down\02-07-2019_DATA_11_11_23\GPS.csv
ON file  ./Trails\down\02-07-2019_DATA_12_06_05\GPS.csv
ON file  ./Trails\down\03-07-2019\GPS.csv
ON file  ./Trails\down\03-07-2019_DATA_09_56_40\GPS.csv
ON file  ./Trails\down\03-07-2019_DATA_11_52_11\GPS.csv
ON file  ./Trails\down\03-07-2019_DATA_18_04_00\GPS.csv
ON file  ./Trails\down\04-07-2019\GPS.csv
ON file  ./Trails\down\04-07-2019_DATA_19_23_45\GPS.csv
ON file  ./Trails\down\05-07-2019\GPS.csv
ON file  ./Trails\down\05-07-2019_DATA_09_21_15\GPS.csv
ON file  ./Trails\down\05-07-2019_DATA_10_56_28\GPS.csv
ON file  ./Trails\down\05-07-2019_DATA_11_49_44\GPS.csv
ON file  ./Trails\down\05-07-2019_DATA_17_01_55\GPS.csv
ON file  ./Trails\down\05-07-2019_DATA_18_56_42\GPS.csv
ON file  ./Trails\down\30-06-2019\GPS.csv
ON file  ./Trails\up

In [21]:
df=pd.DataFrame(speed_with_tz,columns=['speed','tz'])

In [24]:
df.groupby('tz')['speed'].median().mean()

7.819137665037081